In [2]:
#to accurately train model
#i'll simply extract digits from sample sudoku images from web
#augment and use them to train you cnn

In [1]:
!pip install opencv-python
!pip install tensorflow==2.15
!pip install imutils
!pip install scikit-image


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [15]:
import numpy as np
import cv2
import tensorflow as tf
import imutils
import imutils.perspective
from skimage.segmentation import clear_border
import glob

In [22]:
i = 1

In [36]:
for image_path in glob.glob("train_images/*.png"):
    #read image as grayscale

    gray_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    #blur it slightly
    image = gray_image
    blurred = cv2.GaussianBlur(image, (7,7),3)

    # apply adaptive thresholding and then invert the threshold map
    thresh = cv2.adaptiveThreshold(
        blurred, 
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY, 
        11, 
        2
    )
    thresh = cv2.bitwise_not(thresh)

    # find contours in the thresholded image and sort them by size in
        # descending order
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    # initialize a contour that corresponds to the puzzle outline
    puzzleCnt = None
    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # if our approximated contour has four points, then we can
        # assume we have found the outline of the puzzle
        if len(approx) == 4:
            puzzleCnt = approx
            break

    if(len(puzzleCnt) == 0):
        break    #no contour has been found
    
    # apply a four point perspective transform to both the original
    # image and grayscale image to obtain a top-down bird's eye view
    # of the puzzle

    warped = imutils.perspective.four_point_transform(image, puzzleCnt.reshape(4, 2))
    
    print(warped.shape)

    cells = []

    h, w = warped.shape
    cell_h, cell_w = h//9, w//9

    for i in range(9):
        for j in range(9):
            cell = warped[i*cell_h:(i+1)*cell_h, j*cell_w:(j+1)*cell_w]        
            cells.append(cell)


    for row in cells:
        for cell in row:
            # apply automatic thresholding to the cell and then clear any
            # connected borders that touch the border of the cell
            thresh = cv2.threshold(cell, 0, 255,
                cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            thresh = clear_border(thresh)

            # find contours in the thresholded cell
            cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
                cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)


            # if no contours were found than this is an empty cell
            if len(cnts) == 0:
                pass
            
            else:
                # otherwise, find the largest contour in the cell and create a
                # mask for the contour
                c = max(cnts, key=cv2.contourArea)
                mask = np.zeros(thresh.shape, dtype="uint8")
                cv2.drawContours(mask, [c], -1, 255, -1)

                # compute the percentage of masked pixels relative to the total
                # area of the image
                (h, w) = thresh.shape
                percentFilled = cv2.countNonZero(mask) / float(w * h)

                # if less than 3% of the mask is filled then we are looking at
                # noise and can safely ignore the contour
                if percentFilled < 0.03:
                    pass

                else:
                    # apply the mask to the thresholded cell
                    digit = cv2.bitwise_and(thresh, thresh, mask=mask)
                    cv2.imwrite(f"digits/img{i}.png", digit)
                    i += 1

(633, 633)
(112, 5)


AttributeError: 'NoneType' object has no attribute 'shape'